<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/1_train_bert_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## finetuning de BETO para miltiples etiquetas


tomo este ejemplo:


https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb

In [1]:
!pip install datasets seaborn
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [2]:
# no lograba bajar el dataset porque
# me fallaba bajar archivos de hugginface y tenia que actualizar esto (JUAN MA me ayudo)
#pip install -U datasets

In [1]:
from datasets import load_dataset
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

(1) Dataset de etiquetado

https://huggingface.co/datasets/piuba-bigdata/contextualized_hate_speech

Este es el dataset original de etiquetado. Contiene los siguientes campos:

ID : identificador

TITLE: título de la noticia

TEXT: texto del tweet del comentario al Tweet original

CONTEXT: tweet original

HATEFUL: 1 hateful, 0 not hateful

BODY: texto de la noticia

CALLS: si se realizó o no un llamado a la acción

WOMEN: discriminación hacia mujeres

LGBTI: discriminación hacia grupos LGBTI

RACISM: discriminación asociada a racismo

CLASS: discriminación vinculada a pobreza, villas

POLITICS: odio o violencia vinculada a las ideas políticas

DISABLED: discriminación a las personas con discapacidad

APPEARENCE: discriminación en base al aspecto físico (incluye especialmente la gordofobia)

CRIMINAL: discriminación hacia personas en el sistema penal o similares



In [2]:
# para ver las versiones de las librerias
#pip freeze

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from datasets import load_dataset

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")



In [6]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [8]:
##Esto era para levantarlo desde parquet cuando no pude hacerlo con HUGGINFACE
## cargo  train, test y dev por separado
##ds1_train=pd.read_parquet('train/train-00000-of-00001.parquet', engine='pyarrow')
###ds1_train

El dataset 1 contiene 56869 tw etiquetados.
El dataset se dividió en 36420 de train, test:11343 y vaLidacion:9106

In [8]:
ds1["train"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 36420
})

In [9]:
#example1= ds1_train[30355]

example1= ds1["train"][21255]
example1

{'id': 368484,
 'title': 'Video indignante: piba violó la cuarentena y viajó en el baúl de un taxi para ver a un chico',
 'text': '@usuario Estilo Aníbal Fernández',
 'context_tweet': 'Video indignante: piba violó la cuarentena y viajó en el baúl de un taxi para ver a un chico #LogróPasarControlPolicial #EnPuentePueyrredón #FilmóTodo #LoPublicóEnRedesSociales #ComoSiFueraUnaHazaña #Tonta #PoneVidasEnRiesgo #TodoPorUnChongo #MiráEsoChacal https://t.co/ZNlLNNJGaj',
 'HATEFUL': 0,
 'body': 'Si eran pocos los irresponsables que violaban la cuarentena dispuesta por el Gobierno nacional semanas atrás, ahora apareció una joven para engrosar la lista de hechos insólitos que avergüenzan a la sociedad. Es que la chica de unos 25 años cruzó el control policial impuesto en el acceso a Capital Federal desde el partido de Avellaneda, adentro del baúl de un taxi, y para consagrar su hazaña, grabó un video que subió a las redes sociales en el momento del hecho.\n\nEl hecho tuvo lugar en la madrugada d

## Aplicando distintos modelos para entrenar. Usando el siguient tutorial
Uso  de base el siguiente tutorial para hacer el finetuning:
https://huggingface.co/blog/sentiment-analysis-python
y este:
https://huggingface.co/docs/transformers/training


La idea es utilizar la librería tranbsformers de huggingface para hacerle finetuning a un BETO original, es decir la version de BERT pre-entrenada con español de GOOGLE pero que no haya sido reentrenado con ningún otro dataset.
Aqui para usar transformers de hugginface
https://github.com/huggingface/transformers?tab=readme-ov-file



In [10]:
import torch
torch.cuda.is_available()

True

### 1. pre-process data

In [11]:
 ds1['train'][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

## PREPROCESAMIENTO


Preparo los datasets solo con las etiquetas que necesito para la multiclasificación.

In [12]:
ds_hate=ds1.select_columns(["id","text","HATEFUL","CALLS","WOMEN","LGBTI","RACISM","CLASS","POLITICS","DISABLED","APPEARANCE","CRIMINAL"])
ds_hate

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

Transformo las etiquetas originales en labels


In [13]:

labels = [label for label in ds_hate['train'].features.keys() if label not in ['id', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['HATEFUL',
 'CALLS',
 'WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

La siguiente función va a preparar los datos. Aplica el tokenizado usando la función auto-tokenizer del modelo BETO y y construye una matriz de labels

In [14]:

from transformers import AutoTokenizer
import numpy as np

#uso este tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")


#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

Aplico la funcion al dataset de train

In [16]:
ds_hate

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [15]:
encoded_dataset = ds_hate.map(preprocess_data, batched=True, remove_columns=ds_hate['train'].column_names)

Map:   0%|          | 0/11343 [00:00<?, ? examples/s]

Pruebo el encoded

In [17]:
example =  encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [18]:
tokenizer.decode(example['input_ids'])

'[CLS] @ usuario uno menos [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [19]:
example['labels']

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [20]:

[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['HATEFUL', 'CRIMINAL']

In [21]:

encoded_dataset.set_format("torch")

## Defino el modelo y entreno

In [22]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparo los parámetros para ele entrenamiento

F1 score is a machine learning evaluation metric that measures a model’s accuracy. It combines the precision and recall scores of a model.

The accuracy metric computes how many times a model made a correct prediction across the entire dataset. This can be a reliable metric only if the dataset is class-balanced; that is, each class of the dataset has the same number of samples.

In [23]:
batch_size = 16
metric_name = "f1"

In [27]:

##en este caso utilizo una combinacion de las opciones de ambos ejemplos para mejorar el performance

from transformers import TrainingArguments, Trainer

repo_name = "beto_hate_speech_ar_multiclasif"

args = TrainingArguments(
	 output_dir=repo_name,
   evaluation_strategy = "epoch",
   learning_rate=2e-5,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   num_train_epochs=5,
   weight_decay=0.01,
   metric_for_best_model=metric_name,
   save_strategy="epoch",
   push_to_hub=True,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



Funcion para computar las metricas


In [25]:
##FUNCION PARA COMPUTAR LAS METRIcas

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Revision de parametros de TRAINER
https://huggingface.co/docs/transformers/main_classes/trainer

Definicipon de parametros
https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.TrainingArguments



In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [30]:
# Train pre-trained model
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.094000,0.088837,0.541412,0.706348,0.863793
2,0.071600,0.090062,0.598739,0.772660,0.849775
3,0.048800,0.095878,0.587933,0.746158,0.863881
4,0.034000,0.108959,0.592564,0.758306,0.860443
5,0.024800,0.113230,0.590630,0.760274,0.858415


TrainOutput(global_step=11385, training_loss=0.057423003661198, metrics={'train_runtime': 4413.3262, 'train_samples_per_second': 41.261, 'train_steps_per_second': 2.58, 'total_flos': 1.19789911696896e+16, 'train_loss': 0.057423003661198, 'epoch': 5.0})

In [31]:
trainer.evaluate()

{'eval_loss': 0.11322988569736481,
 'eval_f1': 0.5906298003072197,
 'eval_roc_auc': 0.7602737916129998,
 'eval_accuracy': 0.8584148814246672,
 'eval_runtime': 76.7473,
 'eval_samples_per_second': 147.797,
 'eval_steps_per_second': 9.238,
 'epoch': 5.0}

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


Voy a probar el modelo entrenado y guardado

In [32]:
#text = "I'm happy I can finally train a model for multi-label classification"


data = ["hay que matarlos a todos", "Vamos a la plaza a marchar"]
specific_model = pipeline(model="natdebandi/beto_hate_speech_ar_multiclasif")
#specific_model(data)


encoding = tokenizer(data, return_tensors="pt")
encoding = {k: v.to(trainer.specific_model.device) for k,v in encoding.items()}

outputs = trainer.specific_model(**encoding)

NameError: name 'pipeline' is not defined

In [ ]:
from transformers import pipeline
data = ["hay que matarlos a todos", "Vamos a la plaza a marchar"]
specific_model = pipeline(model="natdebandi/beto-hate_speech_ar_bin")
specific_model(data)

[{'label': 'LABEL_1', 'score': 0.951015830039978},
 {'label': 'LABEL_0', 'score': 0.9810741543769836}]

In [ ]:
data = ["hay que matarlos a todos"]


pipe("sos un hijo de puta")

[{'label': 'LABEL_0', 'score': 0.967201828956604}]